<img src="https://lh3.google.com/u/0/d/139J1S3vUM-ToCSZgGeZciZhShC1tY4Jo=w1366-h626-iv1">

# Importaciones

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import os
import cv2 as cv
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gc
import tensorflow as tf

# Procedimientos

## Cargamos los datos

In [3]:
x_load = np.load('/content/drive/My Drive/Proyecto IA2/numpys/x10.npy')

In [4]:
print(x_load.shape)

(10000, 224, 224, 3)


In [5]:
y_load = np.load('/content/drive/My Drive/Proyecto IA2/numpys/y10.npy')
print(y_load.shape)
print(y_load)

(10000,)
[0 0 0 ... 1 1 1]


In [6]:
np.unique(y_load)

array([0, 1])

## Dividimos la prueba y el entrenamiento

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x_load, y_load, test_size=0.08)

In [8]:
X_train = X_train.reshape([-1, 224, 224, 3])#Agrega un indice que exige keras (-1), 224x224, por tres canales
X_test = X_test.reshape([-1, 224, 224, 3])

X_train = X_train.astype('float32')#Lo pasamos a flotantes
X_test = X_test.astype('float32')

X_train /= 255 #Escala entre 0-1
X_test /= 255


In [ ]:
gc.collect()

28

In [ ]:
del x_load

In [ ]:
del y_load

## Arquitecturas

### VGG19

In [9]:
model_A = tf.keras.applications.VGG19(weights='imagenet')

574717952/574710816 [==============================] - 4s 0us/step


In [10]:
output = model_A.layers[-3].output
inicializador = tf.keras.initializers.lecun_uniform(seed=None)

x = tf.keras.layers.Dense(100, activation='tanh')(output)
x = tf.keras.layers.Dense(50, activation='tanh')(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(model_A.input, x)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
opt = tf.keras.optimizers.SGD(lr=0.00001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

288/288 [==============================] - 163s 565ms/step - loss: 0.6643 - accuracy: 0.6390 - val_loss: 0.5929 - val_accuracy: 0.7962


In [13]:
opt = tf.keras.optimizers.Adam(lr=0.00001)
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

288/288 [==============================] - 165s 572ms/step - loss: 0.2025 - accuracy: 0.9308 - val_loss: 0.1109 - val_accuracy: 0.9638


In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.00000001)
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

288/288 [==============================] - 75s 259ms/step - loss: 0.1118 - accuracy: 0.9608 - val_loss: 0.1222 - val_accuracy: 0.9625


In [ ]:
opt = tf.keras.optimizers.Adadelta(lr=0.1)
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

288/288 [==============================] - 76s 263ms/step - loss: 0.1534 - accuracy: 0.9543 - val_loss: 0.1301 - val_accuracy: 0.9588


### Guardamos el modelo

In [15]:
tf.keras.models.save_model(model,'/content/drive/My Drive/Proyecto IA2/Models/vgg19.h5')